In [ ]:
import torch
from transformers import BertTokenizer, BertForMultipleChoice
import logging
logging.basicConfig(level=logging.INFO)
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
from torch.utils.data import DataLoader, Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_folder = "../pretrained_models"
model_name = "bert-base-uncased"
model_path = os.path.join(model_folder, model_name)
learning_rate = 1e-5
batch_size = 32
num_epochs = 8

In [3]:
def get_data_and_preprocess(texts_path, labels_path):
    texts = pd.read_csv(texts_path, header=0, names=['ID', 'Sentence 0', 'Sentence 1'])
    labels = pd.read_csv(labels_path, header=None, names=['ID', 'Answer'])['Answer']
    for i in range(len(texts)):
        if not texts.iloc[i, 1].endswith("."):  # 加句号
            texts.iloc[i, 1] += "."
        if not texts.iloc[i, 2].endswith("."):
            texts.iloc[i, 2] += "."
    return texts, labels

In [4]:
train_texts, train_labels = get_data_and_preprocess(
    '../data/Training Data/subtaskA_data_all.csv',
    '../data/Training Data/subtaskA_answers_all.csv'
)

val_texts, val_labels = get_data_and_preprocess(
    '../data/Dev Data/subtaskA_dev_data.csv',
    '../data/Dev Data/subtaskA_gold_answers.csv'
)

test_texts, test_labels = get_data_and_preprocess(
    '../data/Test Data/subtaskA_test_data.csv',
    '../data/Test Data/subtaskA_gold_answers.csv'
)

# Use the validation dataset as the training dataset
train_texts = pd.concat([train_texts, val_texts], axis=0, ignore_index=True)
train_labels = pd.concat([train_labels, val_labels], axis=0, ignore_index=True)
train_data = pd.concat([train_texts, train_labels], axis=1)
train_data

ID                                         Sentence 0  \
0         0              He poured orange juice on his cereal.   
1         1                                   He drinks apple.   
2         2                             Jeff ran a mile today.   
3         3                              A mosquito stings me.   
4         4                               A niece is a person.   
...     ...                                                ...   
10992   700                      THE TEMPERATURE IS EMOTIONAL.   
10993   355                    Blended up fruit makes a nacho.   
10994  1550                           Dogs falls from the sky.   
10995  1364                   He put a dinosaur in the fridge.   
10996  1257  The cleaner is in charge of the money at the s...   

                                              Sentence 1  Answer  
0                          He poured milk on his cereal.       0  
1                                        He drinks milk.       0  
2                          Jeff ran 100,000 miles today.       1  
3                                    I sting a mosquito.       1  
4                                 A giraffe is a person.       1  
...                                                  ...     ...  
10992                           THE TEMPERATURE IS WARM.       0  
10993                 Blended up fruit makes a smoothie.       0  
10994                           Rain falls from the sky.       0  
10995                    He put a chicken in the fridge.       0  
10996  The cashier is in charge of the money at the s...       0  

[10997 rows x 4 columns]

In [5]:
question = "Which one is against common sense?"
tokenizer = BertTokenizer.from_pretrained(model_path)


train_encodings = tokenizer([question] * 2 * len(train_data),   
                            sum([train_texts.iloc[i, 1:].tolist() for i in range(len(train_texts))], start=[]), 
                            return_tensors="pt", 
                            padding=True)
train_encodings = {key: value.view(len(train_texts), 2, -1) for key, value in train_encodings.items()}


test_encodings = tokenizer([question] * 2 * len(test_texts),   
                            sum([test_texts.iloc[i, 1:].tolist() for i in range(len(test_texts))], start=[]), 
                            return_tensors="pt", 
                            padding=True)
test_encodings = {key: value.view(len(test_texts), 2, -1) for key, value in test_encodings.items()}

In [6]:
class ComVEDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
train_dataset = ComVEDataset(train_encodings, train_labels)
test_dataset = ComVEDataset(test_encodings, test_labels)

In [8]:
model = BertForMultipleChoice.from_pretrained(model_path).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at ../pretrained_models/bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at ../pretrai

In [9]:
# Training
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Start training
model.train()

train_loss = list()
train_accuracies = list()
highest_accuracy = 0
for epoch in range(num_epochs):
    print('Epoch %s/%s' % (epoch + 1, num_epochs))
    
    correct = 0
    count = 0
    epoch_loss = list()
    
    pbar = tqdm(train_loader)
    for batch in pbar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
        loss = outputs["loss"]
        loss.backward()
        optimizer.step()
        
        # make predictions
        predictions = outputs["logits"].argmax(dim=1)

        # count accuracy
        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct * 1.0 / count

        # show progress along with metrics
        pbar.set_postfix({
            'Loss': '{:.3f}'.format(loss.item()),
            'Accuracy': '{:.3f}'.format(accuracy)
        })
        
        # record the loss for each batch
        epoch_loss.append(loss.item())
        
    pbar.close()
    
    # record the loss and accuracy for each epoch
    train_loss += epoch_loss
    train_accuracies.append(accuracy)


Epoch 1/8


  0%|                                                                                                                                   | 0/344 [00:00<?, ?it/s]/tmp/ipykernel_6072/3522555481.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:11<00:00,  4.78it/s, Loss=0.535, Accuracy=0.743]


Epoch 2/8


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:13<00:00,  4.67it/s, Loss=0.302, Accuracy=0.868]


Epoch 3/8


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:12<00:00,  4.72it/s, Loss=0.067, Accuracy=0.971]


Epoch 7/8


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:13<00:00,  4.67it/s, Loss=0.143, Accuracy=0.981]


Epoch 8/8


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [01:14<00:00,  4.62it/s, Loss=0.085, Accuracy=0.984]


In [10]:
# Testing

# Get test data by DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Start testing
model.eval()

with torch.no_grad():
    
    correct = 0
    count = 0
    record = list()
    
    pbar = tqdm(test_loader)
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
        loss = outputs["loss"]
        
        # make predictions
        predictions = outputs["logits"].argmax(dim=1)

        # count accuracy
        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct * 1.0 / count

        # show progress along with metrics
        pbar.set_postfix({
            'loss': '{:.3f}'.format(loss.item()),
            'accuracy': '{:.3f}'.format(accuracy)
        })
    
        # record the results
        record.append((int(labels),int(predictions)))
        
    pbar.close()
    
print("The final accuracy on the test dataset: %s%%" % round(accuracy*100, 4))

  0%|                                                                                                                                  | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_6072/3522555481.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|███████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 69.83it/s, loss=0.000, accuracy=0.883]

The final accuracy on the test dataset: 88.3%
